## Use sensors on the LoRa-E5-Dev

The LoRa-E5-Dev board provides several an:
- lm75a: a temperature sensor

And multiple pins to connect other sensors, e.g. grove sensors like:
- [Grove - IMU 10DOF](https://wiki.seeedstudio.com/Grove-IMU_10DOF_v2.0): integrating
a high performance 9-axis motion tracking module (`mpu9250`) and a barometer sensor (`bmp280`).

and many others like: ccs811, adxl6d3, bme680. See [Sensor Device Drivers](https://doc.riot-os.org/group__drivers__sensors.html)
for a complete list of sensors supported in RIOT.

In this exercise, we propose to read and display the values of the integrated lm75a
and an mpu9250 groove sensor. This will be implemented with shell commands and
synchronization between threads.

- The lm75a temperature value will read on demand via a single shell command, `lm75`
- The mpu9250 accelerometer values will be read continuously every 500ms after calling `mpu start`.
The continuous read will be stopped by calling `mpu stop`

Extending or replacing one of this sensors should be farley easy, by looking
at the drivers documentation and the examples under [tests/drivers](https://github.com/RIOT-OS/RIOT/tree/master/tests)


### Read the lm75a sensor using a shell command

To read the lm75a sensor, we will use the lm75a module driver.

1. In the application [Makefile](Makefile), add the lm75a module to the build:

```Makefile
USEMODULE += lm75a
```

2. In the [main.c](main.c), add the necessary includes for this driver:

```c
#include "lm75.h"
#include "lm75_params.h"
```

**Note:** The files, device descriptor type and function names start with `lm75xxx` because this driver is generic and also works with the lm7525hb and lm7522hb variants.

3. Declare the device descriptor variable:

```c
static lm75_t lm75;
```

4. In the main function, initialize the driver:

```c
    lm75_init(&lm75, &lm75_params[0]);
```

5. Implement the `lm75_handler` function. This function will be called when the shell command uses `lm75` as first argument:

```c
    if (!strcmp(argv[1], "temperature")) {
        int temp = 0;
        lm75_get_temperature(&lm75, &temp);
        printf("Temperature: %d.%dºC\n", temp / 1000, temp % 1000);
    }
    else {
        _lm75_usage(argv[0]);
        return -1;
    }
```

6. Add the `lm75` shell command in the list of available commands (in the `shell_command_t` list before the main function):

```c
    { "lm75", "read the lm75 values", lm75_handler },
```

7. Build and flash the application:

In [ ]:
make flash

8. Open a terminal  and connect to the serial port of the LoRa-E5-Dev and observe the value displayed every 2 seconds:

9. In the RIOT shell, verify that the `lm75` command is working as expected:

You can keep this terminal open while performing the next steps of this notebook.

### Read the mpu9x50 sensor

To read the mpu9x50 sensor in a loop and to start/stop this continuos read from a shell command, we will use 2 things:
- the continuous read will be done in a separate thread `mpu9x50_thread`,
- the start/stop feature will use one of the synchronization mechanisms available in RIOT: a mutex. The idea is make the
mpu9x50 thread try to acquire the mutex before reading the values. If the mutex is already acquired by the main thread,
the mpu9x50 thread will be blocked, trying to acquire it. So the start subcommand of the `mpu` shell will just release the
mutex to let the mpu9x50 thread read the values. After each read, the mpu9x50 thread will release the mutex. The stop
subcommand of the `mpu` shell will just acquire the mutex again to clock the mpu9x50 thread.

To read the mpu9x50 sensor, use the mpu9x50 module driver. In this part of the exercise, the driver is read from a separate background thread.

1. In the application [Makefile](Makefile), add the mpu9x50 module to the build:

```Makefile
USEMODULE += mpu9250
```

2. In the [main.c](main.c), add the necessary includes for this driver:

```c
#include "mpu9x50.h"
#include "mpu9x50_params.h"
```

3. Declare the device descriptor variable:

```c
static mpu9x50_t mpu9x50;
```

4. Declare the mutex used to lock/unlock the mpu9x50 thread. The mutex is initialized locked in order to block the mpu9x50 thread at startup.

```c
static mutex_t mpu_lock = MUTEX_INIT_LOCKED;
```

5. In the main function, initialize the driver and set the sample rate:

```c
    mpu9x50_init(&mpu9x50, &mpu9x50_params[0]);
```

6. In the mpu9x50 thread function, acquire the mutex. This call blocks the thread until the mutex is released by another thread:

```c
        mutex_lock(&mpu_lock);
```

7. In the mpu9x50 thread function, after the mutex is acquired, the accelerometer/magnetometer values can be read:

```c
        mpu9x50_results_t acc_value;
        mpu9x50_read_accel(&mpu9x50, &acc_value);
        printf("Accelerometer x: %i y: %i z: %i\n", acc_value.x_axis,
               acc_value.y_axis, acc_value.z_axis);
```

8. Once the values are read, the mutex must be released:

```c
        mutex_unlock(&mpu_lock);
```

9. Finally we can implement the `start`/`stop` subcommands of the `mpu` shell command `mpu9x50_handler` handler:

```c
    if (!strcmp(argv[1], "start")) {
        mutex_unlock(&mpu_lock);
    }
    else if (!strcmp(argv[1], "stop")) {
        mutex_trylock(&mpu_lock);
    }
    else {
        _mpu9x50_usage(argv[0]);
        return -1;
    }
```
   The `stop` subcommand calls the `mutex_trylock` instead of `mutex_lock` to avoid having the shell blocked when `mpu stop` is called several times in row.

10. Let's build and flash again the application:

In [ ]:
make flash

11. If you kept the previous serial output opened in the terminal, you can test the new `mpu` with the `start` and `stop` subcommands

### Read other sensors

As said at the beginning of this Notebook, other groove sensors can be added to extend this exercise.

In this section, we propose that you extend the application by reading data from the
`bmp280` in the Grove - IMU 10DOF v2.0, as well as extending the `mpu` command to read the
gyroscope values.

The synopsis of the extended commands is the following:
- `mpu` should now also print the compass and gyroscope data
- `bmp` should allow reading the temperature or pressure values and print one or the other

_Note_: the `bmp280`, `lm75a` and `mpu9x250` include a temperature sensor, can you read
all of them? How much do they differ?

To help you with the APIs usage of the corresponding drivers, you can have a look at the following resources:
- [bmx280 API online documentation](https://doc.riot-os.org/group__drivers__bmx280.html)
- The [bmx280 test application code](https://github.com/RIOT-OS/RIOT/tree/master/tests/driver_bmx280) can also be useful
- [mpu9x50 API online documentation](https://doc.riot-os.org/group__drivers__mpu9x50.html)
- The [mpu9x20 test application code](https://github.com/RIOT-OS/RIOT/tree/master/tests/driver_mpu9x50) can also be useful

Also, don't forget to add the corresponding modules in the [Makefile](Makefile). In the
case of the `bmx280` there are multiple versions, so here include `bmp280_i2c`.

Once done, you can rebuild and flash your application.

_Note_: the `bmp280`, `lm75a` and `mpu9x250` include a temperature sensor, can you read
all of them? How much do they differ?

In [ ]:
make flash

**Optional improvement:** the continuous read is always done every 500ms. It is possible to extend the `mpu start` command with an extra parameter to specify the delay between each read:

If unspecified (`argc < 3`) the delay value is set by default to 500ms. If specified (`argc == 3`), the argument (`argv[2]`) can be converted to an int using the `atoi` function. The delay value can be stored in a global static variable (`uint32_t`).

The following command should read the mpu9x50 values every 2s: